In [6]:
import argparse
import os
import torch.backends.cudnn as cudnn
import torch
from torch import nn
from torch.nn import functional as F
import tqdm
import numpy as np
from PIL import Image
from models.sdafnet import SDAFNet_Tryon
from torchvision import transforms, utils
from utils import lpips
from utils.utils import AverageMeter
from torchvision import transforms, utils
from torch.utils import data
from torchvision.utils import save_image

In [9]:
sdafnet = SDAFNet_Tryon(ref_in_channel=6).cuda()
sdafnet = sdafnet.cuda()
sdafnet.load_state_dict(torch.load("ckpt_viton.pt"))
sdafnet.eval()

AssertionError: Torch not compiled with CUDA enabled

In [ ]:
def test(opt, net):
    with torch.no_grad():
        for i, inputs in enumerate(tqdm.tqdm(test_loader)):
            img_names = inputs['img_name']
            cloth_names = inputs['c_name']['paired']
            img = inputs['img'].cuda()
            img_agnostic = inputs['img_agnostic'].cuda() #Masked model image
            pose = inputs['pose'].cuda()
            cloth_img = inputs['cloth']['paired'].cuda()
            img =  F.interpolate(img, size=(256, 192), mode='bilinear')
            cloth_img = F.interpolate(cloth_img, size=(256, 192), mode='bilinear')
            img_agnostic = F.interpolate(img_agnostic, size=(256, 192), mode='bilinear')
            pose = F.interpolate(pose, size=(256, 192), mode='bilinear')
            ref_input = torch.cat((pose, img_agnostic), dim=1)
            tryon_result = net(ref_input, cloth_img, img_agnostic).detach()
            for j in range(tryon_result.shape[0]):
                save_image(tryon_result[j:j+1], os.path.join(opt.save_dir, opt.name, "vis_viton_out", img_names[j]), nrow=1, normalize=True, range=(-1,1))
            if opt.add_compare:
                tryon_result = torch.cat([img_agnostic, cloth_img, tryon_result],2)
                save_image(tryon_result, os.path.join(opt.save_dir, opt.name, "vis_viton_out", img_names[0]), nrow=10, normalize=True, range=(-1,1))
